<a href="https://colab.research.google.com/github/gz2214/Text2Trade/blob/main/code/Bert_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_from_disk
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
import ast
import re
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch.optim as optim
import torch.nn as nn

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dataset_path_block0 = '/content/drive/MyDrive/Text2Trade/data/block0'
dataset_path_block1 = '/content/drive/MyDrive/Text2Trade/data/block1'
dataset_path_block2 = '/content/drive/MyDrive/Text2Trade/data/block2'
dataset_path_block3 = '/content/drive/MyDrive/Text2Trade/data/block3'
dataset_path_block4 = '/content/drive/MyDrive/Text2Trade/data/block4'

block0 = load_from_disk(dataset_path_block0)
block1 = load_from_disk(dataset_path_block1)
block2 = load_from_disk(dataset_path_block2)
block3 = load_from_disk(dataset_path_block3)
block4 = load_from_disk(dataset_path_block4)

blocks = [block0, block1, block2, block3, block4]
block_names = ['block0', 'block1', 'block2', 'block3', 'block4']

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2 # Binary classification (up/down)
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#function to split the text into chunks of tokens - ensure each chunk is less than 512 tokens
def chunk_tokens(text, max_tokens=512):
    tokens = tokenizer.tokenize(text) #tokenize text
    chunks = []
    current_chunk = []
    current_length = 0

    for token in tokens:
        current_chunk.append(token) #append token to current chunk
        current_length += 1

        #once the limit is reached, the current chunk is saved, and a new chunk begins
        if current_length >= max_tokens:
            chunks.append(current_chunk)
            current_chunk = []
            current_length = 0

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [7]:
#process a block of data and create df_train and df_test
def process_blocks(block, block_name):
    train_dataset = block['train']
    test_dataset = block['test']

    train_dates, train_titles, train_labels = [], [], []
    test_dates, test_titles, test_labels = [], [], []

    for item in tqdm(train_dataset, desc=f'Processing Train Data ({block_name})'):
        date = item['Date']
        title = item['Title']
        label = item['Label']
        #tokenizes the titles into chunks
        chunks = chunk_tokens(title)

        for chunk in chunks:
            train_dates.append(date)
            train_titles.append(' '.join(chunk))
            train_labels.append(label)

    for item in tqdm(test_dataset, desc=f'Processing Test Data ({block_name})'):
        date = item['Date']
        title = item['Title']
        label = item['Label']
        chunks = chunk_tokens(title)

        for chunk in chunks:
            test_dates.append(date)
            test_titles.append(' '.join(chunk))
            test_labels.append(label)

    df_train = pd.DataFrame({'Date': train_dates, 'Title': train_titles, 'Label': train_labels})
    df_test = pd.DataFrame({'Date': test_dates, 'Title': test_titles, 'Label': test_labels})

    return df_train, df_test

# Process each block
for block, block_name in zip(blocks, block_names):
    df_train, df_test = process_blocks(block, block_name)
    df_train.to_csv(f'/content/drive/MyDrive/Text2Trade/data/{block_name}_train.csv', index=False)
    df_test.to_csv(f'/content/drive/MyDrive/Text2Trade/data/{block_name}_test.csv', index=False)

Processing Test Data (block4): 100%|██████████| 32/32 [00:01<00:00, 31.29it/s]


In [8]:
train_dataset = block0['train']
num_examples_to_view = 5
example_text = train_dataset.select(range(num_examples_to_view))['Title']

# Train model

Train Bert Classifier using data from block 0 - block 3. No validation required in this step.

In [9]:
"""
def load_train_data(blocks, tokenizer, max_tokens=512, batch_size=16, shuffle=True, train_ratio=0.8):
    # Create empty lists to store data from blocks
    titles = []
    labels = []

    # Merge data from blocks 0 to 3
    for block in blocks[:4]:  # Blocks 0 to 3
        # Extract titles and labels
        block_titles = block['train']['Title']
        block_labels = block['train']['Label']

        # Append titles and labels
        titles.extend(block_titles)
        labels.extend(block_labels)

    # Convert labels to a tensor
    labels = torch.tensor(labels, dtype=torch.long)

    # Tokenize and process titles
    input_ids_list = []
    attention_masks_list = []

    for title in titles:
        inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=max_tokens)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        input_ids_list.append(input_ids)
        attention_masks_list.append(attention_mask)

    # Find the maximum sequence length
    max_sequence_length = max(input_ids.shape[1] for input_ids in input_ids_list)

    # Pad input_ids and attention_mask tensors to have the same length
    for i in range(len(input_ids_list)):
        input_ids_list[i] = torch.cat([input_ids_list[i], torch.zeros(1, max_sequence_length - input_ids_list[i].shape[1], dtype=torch.long)], dim=1)
        attention_masks_list[i] = torch.cat([attention_masks_list[i], torch.zeros(1, max_sequence_length - attention_masks_list[i].shape[1], dtype=torch.long)], dim=1)

    # Stack input_ids tensors and attention_mask tensors along dim=0
    input_ids = torch.stack(input_ids_list)
    attention_masks = torch.stack(attention_masks_list)

    # Split data into train and validation sets based on train_ratio
    train_size = int(train_ratio * len(labels))
    val_size = len(labels) - train_size

    train_dataset = TensorDataset(input_ids[:train_size], attention_masks[:train_size], labels[:train_size])
    val_dataset = TensorDataset(input_ids[train_size:], attention_masks[train_size:], labels[train_size:])

    # Create DataLoader for train and validation sets
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, val_dataloader

train_dataloader, val_dataloader = load_train_data(blocks, tokenizer, batch_size=16, shuffle=True)
"""

"\ndef load_train_data(blocks, tokenizer, max_tokens=512, batch_size=16, shuffle=True, train_ratio=0.8):\n    # Create empty lists to store data from blocks\n    titles = []\n    labels = []\n\n    # Merge data from blocks 0 to 3\n    for block in blocks[:4]:  # Blocks 0 to 3\n        # Extract titles and labels\n        block_titles = block['train']['Title']\n        block_labels = block['train']['Label']\n\n        # Append titles and labels\n        titles.extend(block_titles)\n        labels.extend(block_labels)\n\n    # Convert labels to a tensor\n    labels = torch.tensor(labels, dtype=torch.long)\n\n    # Tokenize and process titles\n    input_ids_list = []\n    attention_masks_list = []\n\n    for title in titles:\n        inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=max_tokens)\n        input_ids = inputs['input_ids']\n        attention_mask = inputs['attention_mask']\n        input_ids_list.append(input_ids)\n        attention_masks

block0 - block4['train']

In [10]:
def load_train_data(blocks, tokenizer, max_tokens=512, batch_size=16, shuffle=True, train_ratio=0.8):
    # Create empty lists to store data from blocks
    titles = []
    labels = []

    # Merge data from blocks 0 to 3
    for block in blocks[:4]:  # Blocks 0 to 3
        # Extract titles and labels
        block_titles = block['train']['Title']
        block_labels = block['train']['Label']

        # Append titles and labels
        titles.extend(block_titles)
        labels.extend(block_labels)

    # Merge data from block4['train']
    block4_train_titles = blocks[4]['train']['Title']
    block4_train_labels = blocks[4]['train']['Label']

    # Append titles and labels
    titles.extend(block4_train_titles)
    labels.extend(block4_train_labels)

    # Convert labels to a tensor
    labels = torch.tensor(labels, dtype=torch.long)

    # Tokenize and process titles
    input_ids_list = []
    attention_masks_list = []

    for title in titles:
        inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=max_tokens)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        input_ids_list.append(input_ids)
        attention_masks_list.append(attention_mask)

    # Find the maximum sequence length
    max_sequence_length = max(input_ids.shape[1] for input_ids in input_ids_list)

    # Pad input_ids and attention_mask tensors to have the same length
    for i in range(len(input_ids_list)):
        input_ids_list[i] = torch.cat([input_ids_list[i], torch.zeros(1, max_sequence_length - input_ids_list[i].shape[1], dtype=torch.long)], dim=1)
        attention_masks_list[i] = torch.cat([attention_masks_list[i], torch.zeros(1, max_sequence_length - attention_masks_list[i].shape[1], dtype=torch.long)], dim=1)

    # Stack input_ids tensors and attention_mask tensors along dim=0
    input_ids = torch.stack(input_ids_list)
    attention_masks = torch.stack(attention_masks_list)

    # Split data into train and validation sets based on train_ratio
    train_size = int(train_ratio * len(labels))
    val_size = len(labels) - train_size

    train_dataset = TensorDataset(input_ids[:train_size], attention_masks[:train_size], labels[:train_size])
    val_dataset = TensorDataset(input_ids[train_size:], attention_masks[train_size:], labels[train_size:])

    # Create DataLoader for train and validation sets
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, val_dataloader

train_dataloader, val_dataloader = load_train_data(blocks, tokenizer, batch_size=16, shuffle=True)


In [11]:
# Iterate through the dataloader to view the first batch
for batch in train_dataloader:
    input_ids, attention_mask, labels = batch
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
    break  # Stop after viewing the first batch

Input IDs: tensor([[[  101,  1047,  2527,  ...,  3796,  1048,   102]],

        [[  101,  5899,  2003,  ...,  1998,  2062,   102]],

        [[  101, 29433,  1005,  ...,  1521,  1055,   102]],

        ...,

        [[  101, 15768,  4139,  ...,  2006,  1029,   102]],

        [[  101, 28517,  2063,  ...,  2539, 28896,   102]],

        [[  101, 26060, 17394,  ..., 17404,  4897,   102]]])
Attention Mask: tensor([[[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1]],

        ...,

        [[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1]]])
Labels: tensor([1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0])


In [12]:
# Define the optimizer with weight decay
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

#we want to leave the last block as "test" after fine tuning.
epochs = 60
# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [13]:
for epoch in range(epochs):
  # ========================================
  #               Training
  # ========================================

  print(f'Epoch {epoch + 1}/{epochs}')
  print('Training...')

  avg_train_loss=[]
  total_train_loss = 0
  model.train()

  for step, batch in enumerate(train_dataloader):
    if step % 40 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    b_input_ids = batch[0].squeeze(1)
    b_input_mask = batch[1]
    b_labels = batch[2]

    #print("Shape of input_ids:", b_input_ids.shape)

    model.zero_grad()
    outputs = model(input_ids = b_input_ids, attention_mask=b_input_mask)

    #loss = outputs.loss
    loss = nn.CrossEntropyLoss()(outputs.logits, b_labels)
    logits = outputs.logits
    total_train_loss += loss.item()

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

  avg_train_loss = total_train_loss / len(train_dataloader)
  print("Average training loss: {0:.2f}".format(avg_train_loss))

Epoch 1/60
Training...
Average training loss: 0.70
Epoch 2/60
Training...
Average training loss: 0.69
Epoch 3/60
Training...
Average training loss: 0.68
Epoch 4/60
Training...
Average training loss: 0.65
Epoch 5/60
Training...
Average training loss: 0.58
Epoch 6/60
Training...
Average training loss: 0.50
Epoch 7/60
Training...
Average training loss: 0.35
Epoch 8/60
Training...
Average training loss: 0.24
Epoch 9/60
Training...
Average training loss: 0.16
Epoch 10/60
Training...
Average training loss: 0.14
Epoch 11/60
Training...
Average training loss: 0.11
Epoch 12/60
Training...
Average training loss: 0.07
Epoch 13/60
Training...
Average training loss: 0.03
Epoch 14/60
Training...
Average training loss: 0.03
Epoch 15/60
Training...
Average training loss: 0.03
Epoch 16/60
Training...
Average training loss: 0.01
Epoch 17/60
Training...
Average training loss: 0.00
Epoch 18/60
Training...
Average training loss: 0.01
Epoch 19/60
Training...
Average training loss: 0.00
Epoch 20/60
Training.

# Test

In [14]:
def load_data_for_testing(blocks, tokenizer, max_tokens=512, batch_size=16, shuffle=True):
    # Create empty lists to store data
    titles = []
    labels = []
    input_ids_list = []  # Add this line
    attention_masks_list = []  # Add this line

    # Merge data from blocks 0 to 4 for training
    for block in blocks[:5]:  # Blocks 0 to 4
        # Extract titles and labels
        block_titles = block['train']['Title']
        block_labels = block['train']['Label']

        # Append titles and labels
        titles.extend(block_titles)
        labels.extend(block_labels)

    # Convert labels to a tensor
    labels = torch.tensor(labels, dtype=torch.long)

    # Tokenize and process titles
    for title in titles:
        inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=max_tokens)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        input_ids_list.append(input_ids)
        attention_masks_list.append(attention_mask)

    # Find the maximum sequence length
    max_sequence_length = max(input_ids.shape[1] for input_ids in input_ids_list)

    # Pad input_ids and attention_mask tensors to have the same length
    for i in range(len(input_ids_list)):
        input_ids_list[i] = torch.cat([input_ids_list[i], torch.zeros(1, max_sequence_length - input_ids_list[i].shape[1], dtype=torch.long)], dim=1)
        attention_masks_list[i] = torch.cat([attention_masks_list[i], torch.zeros(1, max_sequence_length - attention_masks_list[i].shape[1], dtype=torch.long)], dim=1)

    # Stack input_ids tensors and attention_mask tensors along dim=0
    input_ids = torch.stack(input_ids_list)
    attention_masks = torch.stack(attention_masks_list)

    # Create DataLoader for training data
    train_dataset = TensorDataset(input_ids, attention_masks, labels)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

    # Extract day-level labels and titles for testing
    test_dates = blocks[4]['test']['Date']
    test_titles = blocks[4]['test']['Title']
    test_labels = blocks[4]['test']['Label']

    # Tokenize and process titles for testing
    test_input_ids_list = []
    test_attention_masks_list = []

    for title in test_titles:
        inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=max_tokens)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        test_input_ids_list.append(input_ids)
        test_attention_masks_list.append(attention_mask)

    # Find the maximum sequence length for testing data
    max_test_sequence_length = max(input_ids.shape[1] for input_ids in test_input_ids_list)

    # Pad input_ids and attention_mask tensors for testing data to have the same length
    for i in range(len(test_input_ids_list)):
        test_input_ids_list[i] = torch.cat([test_input_ids_list[i], torch.zeros(1, max_test_sequence_length - test_input_ids_list[i].shape[1], dtype=torch.long)], dim=1)
        test_attention_masks_list[i] = torch.cat([test_attention_masks_list[i], torch.zeros(1, max_test_sequence_length - test_attention_masks_list[i].shape[1], dtype=torch.long)], dim=1)

    # Stack input_ids tensors and attention_mask tensors for testing data along dim=0
    test_input_ids = torch.stack(test_input_ids_list)
    test_attention_masks = torch.stack(test_attention_masks_list)

    # Convert labels for testing data to a tensor
    test_labels = torch.tensor(test_labels, dtype=torch.long)

    # Create DataLoader for testing data
    test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, test_dataloader

train_dataloader, test_dataloader = load_data_for_testing(blocks, tokenizer, batch_size=16, shuffle=True)

In [15]:
num_batches_in_test_dataloader = len(test_dataloader)
print("Number of batches in test_dataloader:", num_batches_in_test_dataloader)

batch_size = 16
total_examples_in_test_dataloader = num_batches_in_test_dataloader * batch_size
print("Total number of examples in test_dataloader:", total_examples_in_test_dataloader)

Number of batches in test_dataloader: 2
Total number of examples in test_dataloader: 32


In [16]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, average_precision_score

Each day has different number of chunks

In [17]:
def calculate_num_chunks_per_day(dataset):
    # Convert the dataset to a DataFrame
    df = pd.DataFrame(dataset)

    # Group by unique "Date" and count the number of chunks (labels) per day
    num_chunks_per_day = df.groupby("Date")["Label"].count().values

    return num_chunks_per_day

block4_test = pd.read_csv("/content/drive/MyDrive/Text2Trade/data/block4_test.csv")  # Replace "path_to_block4_test.csv" with the actual file path
num_chunks_per_day = calculate_num_chunks_per_day(block4_test)
num_chunks_per_day

array([ 3,  3,  9,  3,  3,  3,  3,  9,  3,  3,  3,  3,  9,  3,  4,  3,  4,
        9,  3,  3,  4,  3,  9,  4,  3,  3,  3, 10,  3,  3,  3,  4])

In [18]:
day_predictions = [] #store predictions (logits) for each chunk of text
day_probabilities = [] #store probability of class 1 for each chunk
day_ground_truth_labels = [] #store ground truth labels for each chunk
num_examples_per_day = None

# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    for batch in test_dataloader:
        #extract inputs for a day (chunks of text)
        day_input_ids = batch[0].squeeze(1)
        day_attention_mask = batch[1]
        day_labels = batch[2]

        outputs = model(input_ids=day_input_ids, attention_mask=day_attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)

        #aggregate predictions for the day
        day_predictions.append(torch.argmax(logits, dim=1))
        day_probabilities.append(probabilities[:, 1])  # Probability of class 1

        # Ground truth labels for the day
        day_ground_truth_labels.append(day_labels)

day_predictions = torch.cat(day_predictions)
print("Shape of day_predictions:", day_predictions.shape)
day_probabilities = torch.cat(day_probabilities)
day_ground_truth_labels = torch.cat(day_ground_truth_labels)

#majority vote for each day
day_final_labels = []
current_index = 0

for num_chunks in num_chunks_per_day:
    day_predictions_i = day_predictions[current_index:current_index + num_chunks]
    day_final_label = 1 if (day_predictions_i.sum() > (num_chunks / 2)) else 0
    day_final_labels.extend([day_final_label] * num_chunks)
    current_index += num_chunks

#probability of class 1 for each day
day_probabilities = [day_probabilities[i:i + num_chunks] for i, num_chunks in enumerate(num_chunks_per_day)]
day_avg_probabilities = [torch.mean(prob) for prob in day_probabilities]

#ensure that day_final_labels and day_ground_truth_labels have the same length
day_final_labels = day_final_labels[:len(day_ground_truth_labels)]

# Calculate evaluation metrics
accuracy = accuracy_score(day_ground_truth_labels, day_final_labels)
f1 = f1_score(day_ground_truth_labels, day_final_labels)
precision = precision_score(day_ground_truth_labels, day_final_labels)
map = average_precision_score(day_ground_truth_labels, day_final_labels)

#print("Length of day_ground_truth_labels:", len(day_ground_truth_labels))
#print("Length of day_final_labels:", len(day_final_labels))
print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Mean Average Precision: {map:.2f}")

Shape of day_predictions: torch.Size([32])
Accuracy: 0.47
F1 Score: 0.54
Precision: 0.48
Mean Average Precision: 0.49


Save output into a text file

In [19]:
metrics_dict = {
    "Precision": precision,
    "Mean Average Precision": map,
    "F1 Score": f1,
    "Accuracy": accuracy
}

file_name = "/content/drive/MyDrive/Text2Trade/results/BERT_classifier_eval.txt"

with open(file_name, 'w') as file:
    for metric_name, metric_value in metrics_dict.items():
        file.write(f"{metric_name}: {metric_value:.6f}\n")

print(f"Metrics saved to {file_name}")

Metrics saved to /content/drive/MyDrive/Text2Trade/results/BERT_classifier_eval.txt


Day-level aggregated outputs (from majority vote):


In [20]:
for i, label in enumerate(day_final_labels):
    print(f"Day {i + 1} Aggregated Label: {label}")

Day 1 Aggregated Label: 0
Day 2 Aggregated Label: 0
Day 3 Aggregated Label: 0
Day 4 Aggregated Label: 0
Day 5 Aggregated Label: 0
Day 6 Aggregated Label: 0
Day 7 Aggregated Label: 1
Day 8 Aggregated Label: 1
Day 9 Aggregated Label: 1
Day 10 Aggregated Label: 1
Day 11 Aggregated Label: 1
Day 12 Aggregated Label: 1
Day 13 Aggregated Label: 1
Day 14 Aggregated Label: 1
Day 15 Aggregated Label: 1
Day 16 Aggregated Label: 1
Day 17 Aggregated Label: 1
Day 18 Aggregated Label: 1
Day 19 Aggregated Label: 1
Day 20 Aggregated Label: 1
Day 21 Aggregated Label: 1
Day 22 Aggregated Label: 1
Day 23 Aggregated Label: 1
Day 24 Aggregated Label: 1
Day 25 Aggregated Label: 1
Day 26 Aggregated Label: 1
Day 27 Aggregated Label: 1
Day 28 Aggregated Label: 0
Day 29 Aggregated Label: 0
Day 30 Aggregated Label: 0
Day 31 Aggregated Label: 0
Day 32 Aggregated Label: 0


In [21]:
output_file_name = "/content/drive/MyDrive/Text2Trade/results/BERT_classifier_daylevel_class.txt"

# Open the file for writing
with open(output_file_name, 'w') as output_file:
    for i, label in enumerate(day_final_labels):
        output_file.write(f"Day {i + 1} Aggregated Label: {label}\n")

# Print a message to confirm that the file has been saved
print(f"Day final labels saved to {output_file_name}")

Day final labels saved to /content/drive/MyDrive/Text2Trade/results/BERT_classifier_daylevel_class.txt


Day-level averaged proabaility (for being 1):

In [22]:
for i, prob in enumerate(day_avg_probabilities):
    print(f"Day {i + 1} Average Probability of 1: {prob:.2f}")

Day 1 Average Probability of 1: 0.33
Day 2 Average Probability of 1: 0.67
Day 3 Average Probability of 1: 0.40
Day 4 Average Probability of 1: 0.33
Day 5 Average Probability of 1: 0.00
Day 6 Average Probability of 1: 0.21
Day 7 Average Probability of 1: 0.54
Day 8 Average Probability of 1: 0.57
Day 9 Average Probability of 1: 0.67
Day 10 Average Probability of 1: 0.63
Day 11 Average Probability of 1: 0.30
Day 12 Average Probability of 1: 0.50
Day 13 Average Probability of 1: 0.62
Day 14 Average Probability of 1: 0.53
Day 15 Average Probability of 1: 0.75
Day 16 Average Probability of 1: 1.00
Day 17 Average Probability of 1: 0.75
Day 18 Average Probability of 1: 0.76
Day 19 Average Probability of 1: 0.67
Day 20 Average Probability of 1: 1.00
Day 21 Average Probability of 1: 0.72
Day 22 Average Probability of 1: 0.62
Day 23 Average Probability of 1: 0.54
Day 24 Average Probability of 1: 0.72
Day 25 Average Probability of 1: 0.67
Day 26 Average Probability of 1: 0.33
Day 27 Average Probab

In [23]:
# Specify the file name for the average probabilities
average_probabilities_file_name = "/content/drive/MyDrive/Text2Trade/results/BERT_classifier_daylevel_prob.txt"

# Open the file for writing
with open(average_probabilities_file_name, 'w') as file:
    # Write each day's average probability in the desired format
    for i, prob in enumerate(day_avg_probabilities):
        file.write(f"Day {i + 1} Average Probability of 1: {prob:.2f}\n")

# Print a message to confirm that the file has been saved
print(f"Average probabilities saved to {average_probabilities_file_name}")


Average probabilities saved to /content/drive/MyDrive/Text2Trade/results/BERT_classifier_daylevel_prob.txt


# NOT IN USE

In [24]:
"""
def load_data(data_path, tokenizer, max_tokens=512, batch_size=16, shuffle=True):
    df = pd.read_csv(data_path)

    # Extract titles, labels, and convert them to lists
    titles = df['Title'].tolist()
    labels = df['Label'].tolist()

    input_ids_list = []
    attention_masks_list = []

    for title in titles:
        #encoding
        inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=max_tokens)

        # Extract input_ids tensor and attention_mask tensor from the BatchEncoding
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        input_ids_list.append(input_ids)
        attention_masks_list.append(attention_mask)

    # Find the maximum sequence length
    max_sequence_length = max(input_ids.shape[1] for input_ids in input_ids_list)

    # Pad input_ids and attention_mask tensors to have the same length
    for i in range(len(input_ids_list)):
        input_ids_list[i] = torch.cat([input_ids_list[i], torch.zeros(1, max_sequence_length - input_ids_list[i].shape[1], dtype=torch.long)], dim=1)
        attention_masks_list[i] = torch.cat([attention_masks_list[i], torch.zeros(1, max_sequence_length - attention_masks_list[i].shape[1], dtype=torch.long)], dim=1)

    # Stack input_ids tensors and attention_mask tensors along dim=0
    input_ids = torch.stack(input_ids_list)
    attention_masks = torch.stack(attention_masks_list)

    # Convert labels to a tensor
    labels = torch.tensor(labels, dtype=torch.long)

    # Create a TensorDataset
    dataset = TensorDataset(input_ids, attention_masks, labels)

    # Create a DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return dataloader
"""

"\ndef load_data(data_path, tokenizer, max_tokens=512, batch_size=16, shuffle=True):\n    df = pd.read_csv(data_path)\n\n    # Extract titles, labels, and convert them to lists\n    titles = df['Title'].tolist()\n    labels = df['Label'].tolist()\n\n    input_ids_list = []\n    attention_masks_list = []\n\n    for title in titles:\n        #encoding\n        inputs = tokenizer(title, return_tensors='pt', padding=True, truncation=True, max_length=max_tokens)\n\n        # Extract input_ids tensor and attention_mask tensor from the BatchEncoding\n        input_ids = inputs['input_ids']\n        attention_mask = inputs['attention_mask']\n\n        input_ids_list.append(input_ids)\n        attention_masks_list.append(attention_mask)\n\n    # Find the maximum sequence length\n    max_sequence_length = max(input_ids.shape[1] for input_ids in input_ids_list)\n\n    # Pad input_ids and attention_mask tensors to have the same length\n    for i in range(len(input_ids_list)):\n        input_ids_li

In [25]:
"""
num_blocks = 4

# Lists to store train and test DataLoaders for each block
train_dataloaders = []
test_dataloaders = []

# Load data for each block
for i in range(num_blocks):
    train_data_path = f'/content/drive/MyDrive/Text2Trade/data/block{i}_train.csv'
    test_data_path = f'/content/drive/MyDrive/Text2Trade/data/block{i}_test.csv'

    train_dataloader = load_data(train_data_path, tokenizer, batch_size=16, shuffle=True)
    test_dataloader = load_data(test_data_path, tokenizer, batch_size=16, shuffle=False)

    train_dataloaders.append(train_dataloader)
    test_dataloaders.append(test_dataloader)
"""

"\nnum_blocks = 4\n\n# Lists to store train and test DataLoaders for each block\ntrain_dataloaders = []\ntest_dataloaders = []\n\n# Load data for each block\nfor i in range(num_blocks):\n    train_data_path = f'/content/drive/MyDrive/Text2Trade/data/block{i}_train.csv'\n    test_data_path = f'/content/drive/MyDrive/Text2Trade/data/block{i}_test.csv'\n\n    train_dataloader = load_data(train_data_path, tokenizer, batch_size=16, shuffle=True)\n    test_dataloader = load_data(test_data_path, tokenizer, batch_size=16, shuffle=False)\n\n    train_dataloaders.append(train_dataloader)\n    test_dataloaders.append(test_dataloader)\n"

In [26]:
"""
block0_train_dataloader = train_dataloaders[0]

# Iterate through the dataloader to view the first batch
for batch in block0_train_dataloader:
    input_ids, attention_mask, labels = batch
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
    break  # Stop after viewing the first batch
"""

'\nblock0_train_dataloader = train_dataloaders[0]\n\n# Iterate through the dataloader to view the first batch\nfor batch in block0_train_dataloader:\n    input_ids, attention_mask, labels = batch\n    print("Input IDs:", input_ids)\n    print("Attention Mask:", attention_mask)\n    print("Labels:", labels)\n    break  # Stop after viewing the first batch\n'

In [27]:
"""
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

#we want to leave the last block as "test" after fine tuning.
epochs = 4
# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

training_stats = []
block_validation_losses = []
all_validation_losses = []

for i in range(num_blocks):
    # Split the data into training and validation for the current block
    train_dataloader = train_dataloaders[i]
    validation_dataloader = test_dataloaders[i]

    #train your model on the current block's training data
    for epoch_i in range(epochs):
        # ========================================
        #               Training
        # ========================================

        print(f'Epoch {epoch_i + 1}/{epochs}')
        print('Training...')

        total_train_loss = 0
        model.train()

        for step, batch in enumerate(train_dataloader):
            if step % 40 == 0 and not step == 0:
                print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

            b_input_ids = batch[0].squeeze(1)
            b_input_mask = batch[1]
            b_labels = batch[2]

            #print("Shape of input_ids:", b_input_ids.shape)

            model.zero_grad()
            outputs = model(input_ids = b_input_ids,
                            attention_mask=b_input_mask)

            #loss = outputs.loss
            loss = nn.CrossEntropyLoss()(outputs.logits, b_labels)
            logits = outputs.logits
            total_train_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)
        print("  Average training loss: {0:.2f}".format(avg_train_loss))

        # ========================================
        #               Validation
        # ========================================
        print("Running Validation...")

        model.eval()
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        for batch in validation_dataloader:
            b_input_ids = batch[0].squeeze(1)
            b_input_mask = batch[1]
            b_labels = batch[2]

            with torch.no_grad():
                outputs = model(b_input_ids,
                                attention_mask=b_input_mask)

            loss = nn.CrossEntropyLoss()(outputs.logits, b_labels)
            logits = outputs.logits

            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            total_eval_accuracy += accuracy_score(np.argmax(logits, axis=1), label_ids)

        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
        print("  Validation Loss: {0:.2f}".format(avg_val_loss))

        # Append the validation loss to the list for this block
        block_validation_losses.append(avg_val_loss)

    # Calculate the average validation loss for this block across all epochs
    avg_validation_loss_block = sum(block_validation_losses) / len(block_validation_losses)
    print("Average Validation Loss for Block {}: {:.2f}".format(i, avg_validation_loss_block))

    # Append the average validation loss for this block to the list for all blocks
    all_validation_losses.append(avg_validation_loss_block)

# Calculate the overall average validation loss across all blocks and epochs
avg_validation_loss = sum(all_validation_losses) / len(all_validation_losses)
print("Average Validation Loss Across Blocks and Epochs: {:.2f}".format(avg_validation_loss))
"""

'\noptimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)\n\n#we want to leave the last block as "test" after fine tuning.\nepochs = 4\n# Total number of training steps is [number of batches] x [number of epochs].\n# (Note that this is not the same as the number of training samples).\ntotal_steps = len(train_dataloader) * epochs\n\n# Create the learning rate scheduler.\nscheduler = get_linear_schedule_with_warmup(optimizer,\n                                            num_warmup_steps = 0,\n                                            num_training_steps = total_steps)\n\ntraining_stats = []\nblock_validation_losses = []\nall_validation_losses = []\n\nfor i in range(num_blocks):\n    # Split the data into training and validation for the current block\n    train_dataloader = train_dataloaders[i]\n    validation_dataloader = test_dataloaders[i]\n\n    #train your model on the current block\'s training data\n    for epoch_i in range(epochs):\n        # ====================================